In [1]:
import os
import requests
import zipfile
import io

import numpy as np
import pandas as pd
import re
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
data_path_tsv = "data/mq_variants_intensity.tsv"

# May take a while to download
if not os.path.exists(data_path_tsv):
    os.makedirs("data", exist_ok=True)
    r = requests.post(
        "https://proteomics2.ucsd.edu/ProteoSAFe/DownloadResult?view=mq_variants_intensity&task=a19fe3be4bd84d4a80e58d64e14ba1dd",
        stream=True,
    )
    r.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(r.content)) as z:
        for zipinfo in z.infolist():
            if zipinfo.filename.endswith(".tsv"):
                zipinfo.filename = "mq_variants_intensity.tsv"
            z.extract(zipinfo, "data")

In [3]:
mq_variants_df = pd.read_csv(data_path_tsv, sep="\t")

C:\Users\lucal\AppData\Local\Temp\ipykernel_16632\3250928739.py:1: DtypeWarning: Columns (32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,27

In [8]:
mq_variants_df.head()

,rowid,ccms_row_id,Variant,Variant ID,Unmod variant,Total,Total- Unmodified sequence,Variants- Unmodified sequence,Proteins,Mass,...,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1_unmod
0,1,1,.IITHPNFNGNTLDNDIM+15.995LIK.,37658,.IITHPNFNGNTLDNDIMLIK.,11683,20735,81,TRYP_PIG,2299.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,.VADPDHDHTGFLTEYVATR.,93378,.VADPDHDHTGFLTEYVATR.,11372,15019,62,sp|P28482-2|MK01_HUMAN;sp|P28482|MK01_HUMAN,2144.0,...,"182,810,000","182,810,000","296,340,000","296,340,000","272,890,000","272,890,000","254,860,000","254,860,000","70,792,000","70,792,000"
2,3,3,.LGEHNIDVLEGNEQFINAAK.,50733,.LGEHNIDVLEGNEQFINAAK.,8878,23098,134,TRYP_PIG,2211.1,...,"152,910,000","152,910,000","313,690,000","313,690,000","187,600,000","187,600,000","313,290,000","313,290,000","204,790,000","204,790,000"
3,4,4,.FRHENIIGINDIIR.,25741,.FRHENIIGINDIIR.,8720,12619,33,sp|P28482-2|MK01_HUMAN;sp|P28482|MK01_HUMAN,1709.9,...,"115,160,000","115,160,000","223,460,000","223,460,000","182,890,000","182,890,000","236,530,000","236,530,000","97,725,000","97,725,000"
4,5,5,.ESESTAGSFSLSVR.,21292,.ESESTAGSFSLSVR.,7995,8328,7,sp|P06239-2|LCK_HUMAN;sp|P06239-3|LCK_HUMAN;sp...,1456.7,...,"19,220,000","19,220,000","11,216,000","11,216,000","12,721,000","12,721,000","12,835,000","12,835,000","8,137,600","8,137,600"


In [6]:
# Filter rows where Proteins column doesn't contain a semicolon
single_protein_variants = mq_variants_df[~mq_variants_df['Proteins'].str.contains(';', na=False)]

# Display the shape of the original and filtered dataframes
print(f"Original dataset shape: {mq_variants_df.shape}")
print(f"Filtered dataset shape: {single_protein_variants.shape}")

# Display first few rows of the filtered dataset
single_protein_variants.head()

Original dataset shape: (83706, 1033)
Filtered dataset shape: (16044, 1033)


,rowid,ccms_row_id,Variant,Variant ID,Unmod variant,Total,Total- Unmodified sequence,Variants- Unmodified sequence,Proteins,Mass,...,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1,_dyn_#Baricitinib 300nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1,_dyn_#Baricitinib 30nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1,_dyn_#Baricitinib 3nM.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1,_dyn_#Baricitinib DMSO.Tech replicate 1 of 1_unmod,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1,_dyn_#Baricitinib PDPD.Tech replicate 1 of 1_unmod
0,1,1,.IITHPNFNGNTLDNDIM+15.995LIK.,37658,.IITHPNFNGNTLDNDIMLIK.,11683,20735,81,TRYP_PIG,2299.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,.LGEHNIDVLEGNEQFINAAK.,50733,.LGEHNIDVLEGNEQFINAAK.,8878,23098,134,TRYP_PIG,2211.1,...,"152,910,000","152,910,000","313,690,000","313,690,000","187,600,000","187,600,000","313,290,000","313,290,000","204,790,000","204,790,000"
5,6,6,.NYLLSLPHK.,68115,.NYLLSLPHK.,7445,11842,41,sp|P28482|MK01_HUMAN,1084.6,...,"290,970,000","290,970,000","477,300,000","477,300,000","363,140,000","363,140,000","43,697,000","43,697,000","182,850,000","182,850,000"
11,12,12,.PM+15.995FIVNTNVPR.,69185,.PMFIVNTNVPR.,5232,10053,12,sp|P14174|MIF_HUMAN,1303.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,13,13,.KLEAAEDIAYQLSR.,44634,.KLEAAEDIAYQLSR.,4958,6905,36,sp|P35232|PHB_HUMAN,1606.8,...,"32,624,000","32,624,000","55,130,000","55,130,000","52,332,000","52,332,000","52,156,000","52,156,000","55,709,000","55,709,000"
